In [1]:
import feather
import json
import pandas as pd
import pprint
import praw

# Display options
pd.set_option('display.max_columns', 500)

In [2]:
# Import client id and secret from file
with open("keys.json", "r") as credentials:
    data = json.load(credentials)
    c_id = data["client_id"]
    c_secret = data["client_secret"]
    u = data["username"]
    p = data["password"]

In [3]:
reddit = praw.Reddit(
    client_id = c_id,
    client_secret = c_secret,
    user_agent = "my user agent",
    username = u,
    password = p,
)

print(reddit.read_only)

False


In [4]:
# Quick look at an example layout
type(reddit.subreddit("learnpython").top("year", limit = 2))

praw.models.listing.generator.ListingGenerator

In [5]:
for submission in reddit.subreddit("learnpython").top("year", limit = 1):
    pprint.pprint(vars(submission))

{'_comments_by_id': {},
 '_fetched': False,
 '_reddit': <praw.reddit.Reddit object at 0x7ff11560ae20>,
 'all_awardings': [{'award_sub_type': 'PREMIUM',
                    'award_type': 'global',
                    'awardings_required_to_grant_benefits': None,
                    'coin_price': 30,
                    'coin_reward': 0,
                    'count': 1,
                    'days_of_drip_extension': 0,
                    'days_of_premium': 0,
                    'description': 'A glittering stamp for a feel-good thing',
                    'end_date': None,
                    'giver_coin_reward': None,
                    'icon_format': None,
                    'icon_height': 2048,
                    'icon_url': 'https://i.redd.it/award_images/t5_22cerq/b9ks3a5k7jj41_WholesomeSealofApproval.png',
                    'icon_width': 2048,
                    'id': 'award_c4b2e438-16bb-4568-88e7-7893b7662944',
                    'is_enabled': True,
                    'is

In [6]:
# As a dataframe
df = pd.DataFrame([ vars(post) for post in reddit.subreddit("learnpython").top("year", limit=10)])
df.head(3)

,comment_limit,comment_sort,_reddit,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,link_flair_richtext,subreddit_name_prefixed,hidden,pwls,link_flair_css_class,downs,top_awarded_type,hide_score,name,quarantine,link_flair_text_color,upvote_ratio,author_flair_background_color,subreddit_type,ups,total_awards_received,media_embed,author_flair_template_id,is_original_content,user_reports,secure_media,is_reddit_media_domain,is_meta,category,secure_media_embed,link_flair_text,can_mod_post,score,approved_by,author_premium,thumbnail,edited,author_flair_css_class,author_flair_richtext,gildings,content_categories,is_self,mod_note,created,link_flair_type,wls,removed_by_category,banned_by,author_flair_type,domain,allow_live_comments,selftext_html,likes,suggested_sort,banned_at_utc,view_count,archived,no_follow,is_crosspostable,pinned,over_18,all_awardings,awarders,media_only,can_gild,spoiler,locked,author_flair_text,treatment_tags,visited,removed_by,num_reports,distinguished,subreddit_id,mod_reason_by,removal_reason,link_flair_background_color,id,is_robot_indexable,report_reasons,author,discussion_type,num_comments,send_replies,whitelist_status,contest_mode,mod_reports,author_patreon_flair,author_flair_text_color,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video,_fetched,_comments_by_id
0,2048,confidence,<praw.reddit.Reddit object at 0x7ff11560ae20>,None,learnpython,**Python 101 2nd Edition** is the latest versi...,t2_4j2wg,False,None,3,False,"As a Gift to the Community, I'm Making my Pyth...",[],r/learnpython,False,6,None,0,None,False,t3_kv61rj,False,dark,0.98,None,public,2036,90,{},None,False,[],None,False,False,None,{},None,False,2036,None,False,,1612966629.0,None,[],"{'gid_1': 25, 'gid_2': 3}",None,True,None,1.610411e+09,text,6,None,None,text,self.learnpython,True,"<!-- SC_OFF --><div class=""md""><p><strong>Pyth...",None,None,None,None,False,False,True,False,False,"[{'giver_coin_reward': None, 'subreddit_id': N...",[],False,True,False,False,None,[],False,None,None,None,t5_2r8ot,None,None,,kv61rj,True,None,driscollis,None,278,True,all_ads,False,[],False,None,/r/learnpython/comments/kv61rj/as_a_gift_to_th...,all_ads,False,https://www.reddit.com/r/learnpython/comments/...,513490,1.610382e+09,4,None,False,False,{}
1,2048,confidence,<praw.reddit.Reddit object at 0x7ff11560ae20>,None,learnpython,https://inventwithpython.com/automateudemy (Th...,t2_369qx,False,None,3,False,"""Automate the Boring Stuff with Python"" Udemy ...",[],r/learnpython,False,6,None,0,None,False,t3_gbp0a1,False,dark,0.99,None,public,1996,9,{},None,False,[],None,False,False,None,{},None,False,1996,None,True,,False,None,[],{'gid_2': 3},None,True,None,1.588391e+09,text,6,None,None,text,self.learnpython,True,"<!-- SC_OFF --><div class=""md""><p><a href=""htt...",None,None,None,None,True,False,True,False,False,"[{'giver_coin_reward': 0, 'subreddit_id': None...",[],False,True,False,False,None,[],False,None,None,None,t5_2r8ot,None,None,,gbp0a1,True,None,AlSweigart,None,168,False,all_ads,False,[],False,None,/r/learnpython/comments/gbp0a1/automate_the_bo...,all_ads,False,https://www.reddit.com/r/learnpython/comments/...,513490,1.588363e+09,6,None,False,False,{}
2,2048,confidence,<praw.reddit.Reddit object at 0x7ff11560ae20>,None,learnpython,About 5 years ago I started to study programmi...,t2_4dgec,False,None,0,False,"After 5 years of learning Python as a hobby, y...",[],r/learnpython,False,6,None,0,None,False,t3_he7vpy,False,dark,0.99,None,public,1798,3,{},None,False,[],None,False,False,None,{},None,False,1798,None,False,,1592886531.0,None,[],{},None,True,None,1.592915e+09,text,6,None,None,text,self.learnpython,False,"<!-- SC_OFF --><div class=""md""><p>About 5 year...",None,None,None,None,True,False,True,False,False,"[{'giver_coin_reward': None, 'subreddit_id': N...",[],False,True,False,False,None,[],False,None,None,None,t5_2r8ot,None,None,,he7vpy,True,None,marcovirtua

In [7]:
# Function to retrieve and store a number of posts from a given subreddit
def get_posts(subreddit, num_posts = 100):
    try:
        # Extract posts and save to dataframe
        df = pd.DataFrame([ vars(post) for post in reddit.subreddit(subreddit).top("year", limit = num_posts) ])
        # Extract the required columns
        cleaned_df = df[['title', 'selftext', 'created_utc', 'author', 'num_comments', 'score', 'upvote_ratio', 'url']]
        # Convert author from Redditor to string object
        cleaned_df.loc[:,'author'] = cleaned_df.loc[:,'author'].astype(str)
        # Save as a feather - heh!
        path = 'data/' + subreddit
        feather.write_dataframe(cleaned_df, path)
    except Exception as e:
        print('Error:', e)

In [8]:
get_posts("changemyview", 1000)

/Users/dulshani/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1675: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [9]:
# TODO:
# CMV: Also whether the post includes a delta